In [7]:
import requests
import pandas as pd
import re
import datetime
import warnings
warnings.filterwarnings('ignore')


def hotpepper(count,keyword,turnover_rate,business_days,hourly_wage,employee,rent,other):
    API_KEY = "d70f5128bf847e1e"
    URL = 'http://webservice.recruit.co.jp/hotpepper/gourmet/v1/'
    body = {
        'key':API_KEY,
        'keyword':keyword,
        'range':2,
        'format':'json',
        'count':count
    }
    #API情報のすべて
    response = requests.get(URL,body).json()
    #shop情報すべて
    df_shop_data = pd.DataFrame(response['results']['shop'])
    #必要エレメント
    element = ["id","name","capacity","open","budget"]

    shop_info_list = []
    for i in range(0,count):
        shop_info = {}
        shop_info['総席数'] = df_shop_data['capacity'][i]

        #営業時間：open}
        s = df_shop_data['open'][i]
        match = re.findall(r'\d+', s)
        #00:00の場合24:00にする
        if match[2] == "0":
            match[2] = "00"

        for a in range(0,4):
            if len(match[a]) == 1:
                match[a] = f"0{match[a]}"
        shop_info['オープン'] = f"{match[0]}:{match[1]}"
        shop_info['クローズ'] = f"{match[2]}:{match[3]}"

        #単価(平均予算ver)：budget_average
        t = df_shop_data['budget'][i]['average'].replace(",","")
        shop_info['単価'] = re.findall(r'\d+', t)[0]

        shop_info_list.append(shop_info)
    df_shop_alt = pd.DataFrame(shop_info_list)

    df_shop_alt['営業時間'] = (pd.to_datetime(df_shop_alt['クローズ']) - pd.to_datetime(df_shop_alt['オープン']))/ datetime.timedelta(hours=1)
    #24時間表記へ書き換え
    n_24 = 0
    for add24 in df_shop_alt['営業時間']:
        if add24 <= 0: 
            df_shop_alt['営業時間'][n_24] = df_shop_alt['営業時間'][n_24] + 24
        n_24 += 1

    #来店人数 = 回転率 * 席数
    df_shop_alt['回転率'] = turnover_rate
    df_shop_alt['来店人数'] =  df_shop_alt['総席数'] * df_shop_alt['回転率']

    ##売り上げ(単価 * 来店人数
    df_shop_alt['営業日数　'] = business_days
    df_shop_alt['売上'] = df_shop_alt['単価'].astype('int') * df_shop_alt['来店人数'] * df_shop_alt['営業日数　']

    #人件費計算
    df_shop_alt['時給'] = hourly_wage
    df_shop_alt['従業員数'] = employee

    df_shop_alt['人件費'] = df_shop_alt['時給'] * df_shop_alt['従業員数'] * df_shop_alt['営業時間']

    ##経費(家賃 + 人件費 + 水道光熱費 + 消耗品) + (原価)
    df_shop_alt['家賃'] = rent
    df_shop_alt['その他'] = other

    df_shop_alt['経費'] = (df_shop_alt['家賃'] + df_shop_alt['人件費'] + df_shop_alt['家賃'] + df_shop_alt['売上']*0.25).astype('int') +df_shop_alt['その他']

    ##利益
    df_shop_alt['利益'] = df_shop_alt['売上'] - df_shop_alt['経費']

    #識別
    df_shop_alt["ID"] = df_shop_data["id"] 
    df_shop_alt["掲載店名"] = df_shop_data["name"]


    re_columns = [ 'ID','掲載店名','利益','売上', '単価','総席数','回転率','来店人数',
                  '経費','人件費', '従業員数','時給', '営業時間','オープン', 'クローズ' , '家賃','その他']
    return df_shop_alt[re_columns]  

In [1]:
import hotpepper

#インプットパラメータ
count = 10
keyword = '心斎橋駅'
turnover_rate = 3
business_days = 23
hourly_wage = 1000
employee = 5
rent = 200000
other = 100000

hotpepper.hotpepper(count,keyword,turnover_rate,business_days,hourly_wage,employee,rent,other)

,ID,掲載店名,利益,売上,単価,総席数,回転率,来店人数,経費,人件費,従業員数,時給,営業時間,オープン,クローズ,家賃,その他
0,J001192324,かつら亭 別館 心斎橋,7202500,10350000,3000,50,3,150,3147500,60000.0,5,1000,12.00,11:00,23:00,200000,100000
1,J000012674,かつら亭 心斎橋店,14965000,20700000,3000,100,3,300,5735000,60000.0,5,1000,12.00,11:00,23:00,200000,100000
2,J000020452,和食居酒屋 楽遊 心斎橋駅近店,4978700,7341600,3800,28,3,84,2362900,27500.0,5,1000,5.50,17:30,23:00,200000,100000
3,J003497820,JAMES CAFE ジェームスカフェ 心斎橋パルコ店,894000,1932000,1000,28,3,84,1038000,55000.0,5,1000,11.00,12:00,23:00,200000,100000
4,J001282767,すし共友丸 南船場店,4763500,7038000,3000,34,3,102,2274500,15000.0,5,1000,3.00,11:30,14:30,200000,100000
5,J003324243,焼肉 肉王,5620000,8280000,3000,40,3,120,2660000,90000.0,5,1000,18.00,18:00,12:00,200000,100000
6,J001262242,RED SNAPPER レッドスナッパー,490000,1380000,2000,10,3,30,890000,45000.0,5,1000,9.00,18:00,03:00,200000,100000
7,J001223516,中華 広東料理 昌和 心斎橋店,7553000,10764000,3000,52,3,156,3211000,20000.0,5,1000,4.00,11:00,15:00,200000,100000
8,J003297406,焼肉 大,515000,1380000,800,25,3,75,865000,20000.0,5,1000,4.00,11:30,15:30,200000,100000
9,J001117077,ネコカフェ キャットテイル Nekocafe Cattail,902750,1932000,2000,14,3,42,1029250,46250.0,5,1000,9.25,11:00,20:15,200000,100000
